In [308]:
# Code credits: Adapted bits and pieces from https://github.com/webdataset/webdataset/blob/master/docs/gettingstarted.ipynb

import sys
sys.path.append('..')

import gc
import json
import os
from itertools import islice
from datetime import datetime
import pytz
from pytz import timezone
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
import skimage.transform as st
import tqdm

import torch
import torch.optim as optim
from torchvision import transforms
import webdataset as wds

from model.selfattn_3d_cnn import *
from model.baseline_3d_cnn import *
from utils.model_utils import *
from utils.model_run import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [309]:
data_dir = '../data'
shards_dir = os.path.join(data_dir, 'shards_new')

# Opening JSON file
with open('../parameters.json') as json_file:
    parameters = json.load(json_file)

batch_size = 4 #parameters['batch_size']
shard_size = 4 #parameters['shard_size']
parameters

{'batch_size': 16, 'shard_size': 16}

In [310]:
urls = [os.path.join(shards_dir, it) for it in os.listdir(shards_dir) if it.endswith('.tar')]

# Try to overfit on smaller data
# urls = urls[:round(len(urls)*0.3)]

# Another shard directory, continued; realize can't use because keys will collide cuz we refreshed...
# shards_dir2 = os.path.join(data_dir, 'shards_new_cont')
# urls += [os.path.join(shards_dir2, it) for it in os.listdir(shards_dir2) if it.endswith('.tar')]


# All the data
# train_urls = urls[:round(len(urls)*0.7)]
# val_urls = urls[round(len(urls)*0.7):round(len(urls)*0.85)]
# test_urls = urls[round(len(urls)*0.85):]

# Smaller data just to run model once
train_urls = urls[:2]
val_urls = urls[2:3]
test_urls = urls[3:]


print("Number of train shards:", len(train_urls))
print("Number of validation shards:", len(val_urls))
print("Number of test shards:", len(test_urls))

Number of train shards: 2
Number of validation shards: 1
Number of test shards: 93


In [311]:
# Create dataset objects
train_iternum = len(train_urls)*shard_size//batch_size
val_iternum = len(val_urls)*shard_size//batch_size
test_iternum = len(test_urls)*shard_size//batch_size

print("Number of iterations per train epoch:", train_iternum)

train_dataset = (
    wds
    .WebDataset(train_urls, length=train_iternum)
    .shuffle(shard_size)
    .decode('torch')
    .to_tuple('volumes.pyd', 'labels.pyd', 'studynames.pyd')
    .batched(batch_size)
#     .map_tuple(pre_transforms, identity, identity)
)
loader_train = torch.utils.data.DataLoader(train_dataset, num_workers=0, batch_size=None) #setting batch_size = None disables batching

val_dataset = (
    wds
    .WebDataset(val_urls, length=val_iternum)
    .shuffle(shard_size)
    .decode('torch')
    .to_tuple('volumes.pyd', 'labels.pyd', 'studynames.pyd')
    .batched(batch_size)
)
loader_val = torch.utils.data.DataLoader(val_dataset, num_workers=0, batch_size=None)

test_dataset = (
    wds
    .WebDataset(test_urls, length=test_iternum)
    .shuffle(shard_size)
    .decode('torch')
    .to_tuple('volumes.pyd', 'labels.pyd', 'studynames.pyd')
    .batched(batch_size)
)
loader_test = torch.utils.data.DataLoader(test_dataset, num_workers=0, batch_size=None)

# for image, target in islice(dataset, 0, 2):
#     print(image.shape)

Number of iterations per train epoch: 2


In [312]:
gc.collect()

520

In [313]:
USE_GPU = True
dtype = torch.float

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
#     dtype = torch.cuda.FloatTensor
else:
    device = torch.device('cpu')

print(device)
print(dtype)

cuda
torch.float32


In [314]:
# Make log directory and checkpoint directory (DIFFERENT DIRECTORY FROM BASELINE)
dir_nm = datetime.now(tz=pytz.utc).astimezone(timezone('US/Pacific')).strftime('%Y-%m-%d_%H-%M-%S')
# dir_nm = "first_mini_c2fc2"
log_dir = os.path.join('../runs_experiment', dir_nm)
os.mkdir(log_dir)
os.mkdir(os.path.join(log_dir, 'Checkpoints'))


# Model, optimizer, criterion
model = baseline_3DCNN(in_num_ch=1)
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
criterion = torch.nn.BCEWithLogitsLoss()

In [315]:
gc.collect()

22

In [316]:
train_loss_dict, val_loss_dict = train(model, optimizer, criterion, loader_train, loader_val, log_dir, device=device, epochs=1, val_every=5)

Epoch 1: 100%|██████████| 2/2 [00:03<00:00,  1.67s/batch, loss=0.667]/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:529: UserWarning: Length of IterableDataset <webdataset.dataset.Processor object at 0x7f63f4a1f710> was reported to be 2 (when accessing len(dataloader)), but 3 samples have been fetched. 
  warnings.warn(warn_msg)
Epoch 1: : 3batch [00:04,  1.52s/batch, loss=0.882]                  /opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:529: UserWarning: Length of IterableDataset <webdataset.dataset.Processor object at 0x7f63f4a1f710> was reported to be 2 (when accessing len(dataloader)), but 4 samples have been fetched. 
  warnings.warn(warn_msg)
Epoch 1: : 4batch [00:06,  1.44s/batch, loss=0.834]/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:529: UserWarning: Length of IterableDataset <webdataset.dataset.Processor object at 0x7f63f4a1f710> was reported to be 2 (when accessing len(dataloader)), but 5 samples ha

torch.Size([4, 40, 512, 512])
torch.Size([4, 40, 512, 512])
torch.Size([4, 40, 512, 512])
torch.Size([4, 40, 512, 512])
Total iteration 5, validation loss = 0.7198


Epoch 1: : 5batch [00:15,  4.19s/batch, loss=0.593]

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:529: UserWarning: Length of IterableDataset <webdataset.dataset.Processor object at 0x7f63f4a1f710> was reported to be 2 (when accessing len(dataloader)), but 6 samples have been fetched. 
  warnings.warn(warn_msg)
Epoch 1: : 6batch [00:16,  3.25s/batch, loss=1.35] /opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:529: UserWarning: Length of IterableDataset <webdataset.dataset.Processor object at 0x7f63f4a1f710> was reported to be 2 (when accessing len(dataloader)), but 7 samples have been fetched. 
  warnings.warn(warn_msg)
Epoch 1: : 7batch [00:17,  2.58s/batch, loss=0.933]/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:529: UserWarning: Length of IterableDataset <webdataset.dataset.Processor object at 0x7f63f4a1f710> was reported to be 2 (when accessing len(dataloader)), but 8 samples have been fetched. 
  warnings.warn(warn_msg)
Epoch 1: : 8batch [00:18,  2.32s/batch, los

In [317]:
# Common errors and how to fix them:

# Error:
#   RuntimeError: running_mean should contain 1 elements not 8
# Fix: One of your batchnorm 3D parameter values is off

# Error:
#   RuntimeError: CUDA out of memory.
# Fix: Make the model / batch size smaller 
# First try to make batch size smaller. Will require longer training time possibly but does not decrease expressivity of model.
# If need to decrease complexity of model, 

# Error:
#   RuntimeError: Given groups=1, weight of size [1, 1, 1, 1, 1], expected input[8, 4, 5, 32, 32] to have 1 channels, but got 4 channels instead
# Fix: Wrong number of in_channels in self attention layer

In [301]:
a = torch.randn(4, 40, 512, 512)
print(a.size())
transforms.Resize(size=(256, 256))(a).size()

torch.Size([4, 40, 512, 512])


torch.Size([4, 40, 256, 256])